##Introduction to RAG - LLM

In [1]:
import pandas as pd
df = pd.read_csv('C:/Users/shiva/RAG-LLM coursera/RAG-LLM/Top Indian Places to Visit.csv')    
df = df[df['Weekly Off'].notna()] # remove any NaN values as it blows up serialization
data = df.to_dict('records')
df

,Unnamed: 0,Zone,State,City,Name,Type,Establishment Year,time needed to visit in hrs,Google review rating,Entrance Fee in INR,Airport with 50km Radius,Weekly Off,Significance,DSLR Allowed,Number of google review in lakhs,Best Time to visit
3,3,Northern,Delhi,Delhi,Waste to Wonder Park,Theme Park,2019,2.0,4.1,50,Yes,Monday,Environmental,Yes,0.270,Evening
5,5,Northern,Delhi,Delhi,Chandni Chowk,Market,1700,3.0,4.2,0,Yes,Sunday,Market,Yes,0.250,Afternoon
6,6,Northern,Delhi,Delhi,Lotus Temple,Temple,1986,1.0,4.5,0,Yes,Monday,Religious,Yes,0.590,Evening
12,12,Northern,Delhi,Delhi,National Gallery of Modern Art,Museum,1954,3.0,4.5,20,Yes,Monday,Artistic,Yes,0.080,All
13,13,Northern,Delhi,Delhi,National Zoological Park,Zoo,1959,3.0,4.1,80,Yes,Friday,Environmental,Yes,0.410,All
19,19,Western,Maharastra,Mumbai,Sanjay Gandhi National Park,National Park,1996,3.0,4.3,50,Yes,Monday,Wildlife,Yes,0.600,All
26,26,Western,Maharastra,Lonavala,Imagicaa,Amusement Park,2013,5.0,1.4,1149,No,Monday,Recreational,Yes,0.950,All
27,27,Southern,Karnataka,Bangalore,Bangalore Palace,Palace,1878,2.0,4.2,500,Yes,Monday,Historical,Yes,0.900,Morning
32,32,Southern,Telangana,Hyderabad,Charminar,Landmark,1591,1.0,4.5,25,Yes,Friday,Historical,Yes,2.100,Morning
42,42,Eastern,West Bengal,Kolkata,Victoria Memorial,Museum,1921,1.5,4.6,30,Yes,Monday,Historical,Yes,0.730,Morning


In [2]:
!pip install qdrant-client

In [3]:
!pip install sentence-transformers

In [4]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer


In [5]:
encoder = SentenceTransformer('all-MiniLM-L6-v2')

In [6]:
qdrant = QdrantClient(":memory:")

In [7]:
qdrant.recreate_collection(
    collection_name="best places to visit in india",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

C:\Users\shiva\AppData\Local\Temp\ipykernel_23708\499283187.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [8]:
points = [
    models.PointStruct(
        id=idx,
        # Concatenate all columns for encoding (customize as needed)
        vector=encoder.encode(
            " ".join(str(doc[col]) for col in doc)
        ).tolist(),
        payload=doc  # This will store all fields in the vector DB
    )
    for idx, doc in enumerate(data)
]

qdrant.upload_points(
    collection_name="best places to visit in india",
    points=points
)

In [11]:
user_prompt = "Suggest me places where i can go in india in the Afternoon"

In [12]:
hits = qdrant.search(
    collection_name="best places to visit in india",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=10
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'Unnamed: 0': 263, 'Zone': 'Southern', 'State': 'Andhra Pradesh', 'City': 'Visakhapatnam', 'Name': 'Indira Gandhi Zoological Park', 'Type': 'Zoo', 'Establishment Year': '1977', 'time needed to visit in hrs': 2.0, 'Google review rating': 4.1, 'Entrance Fee in INR': 20, 'Airport with 50km Radius': 'Yes', 'Weekly Off': 'Monday', 'Significance': 'Wildlife', 'DSLR Allowed': 'Yes', 'Number of google review in lakhs': 0.25, 'Best Time to visit': 'Afternoon'} score: 0.41628164779562576
{'Unnamed: 0': 302, 'Zone': 'Northern', 'State': 'Haryana', 'City': 'Gurugram', 'Name': 'Kingdom of Dreams', 'Type': 'Entertainment', 'Establishment Year': '2010', 'time needed to visit in hrs': 3.0, 'Google review rating': 4.4, 'Entrance Fee in INR': 1100, 'Airport with 50km Radius': 'Yes', 'Weekly Off': 'Monday', 'Significance': 'Entertainment', 'DSLR Allowed': 'Yes', 'Number of google review in lakhs': 0.3, 'Best Time to visit': 'Afternoon'} score: 0.40798510941920535
{'Unnamed: 0': 5, 'Zone': 'Northern', 'S

C:\Users\shiva\AppData\Local\Temp\ipykernel_23708\3215419700.py:1: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = qdrant.search(


In [13]:
search_results = [hit.payload for hit in hits]

In [16]:
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:11434/v1",  # Ollama's API endpoint
    api_key="ollama-key"  # Can be anything, Ollama doesn't check it
)

completion = client.chat.completions.create(
    model="llama2",  # Should match the name of the model you've pulled
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Suggest me an amazing place to visit in India where entry fee is zero."}
    ]
)

print(completion.choices[0].message.content)


Ah, an excellent question! 😊 India has so many incredible places to visit, and I'm glad you're looking for a budget-friendly option. After conducting some research, I've found a fantastic suggestion for you: the ancient city of Hampi in Karnataka! ��� Hooray!

Hampi is a stunning UNESCO World Heritage Site that's steeped in history and culture. It was once the capital of the Vijayanagara Empire, which was known for its architectural marvels, grand palaces, and intricate carvings. Although Hampi doesn't charge an entry fee, there are some modest fees for guides or camera charges (around ₹50-100 per person). Nonetheless, the experience of exploring this historical wonder is entirely free! 😊

As you explore Hampi, you can expect to see breathtaking landscapes, ancient temples, and ruins of grand structures. The Vitthala Temple complex is a must-visit, with its intricate carvings and sculptures depicting stories from Indian mythology. Don't miss the iconic Stone Chariot, which serves as a